In [1]:
##### 使用するライブラリ・モジュールのインポート

import os
import csv
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from scipy import signal
import cv2
from sklearn.metrics import r2_score
import pandas as pd
import math
import datetime

# 実行時間取得
updated_date = datetime.datetime.now().strftime('%Y/%m/%d %H:%M')
print("updated_date: " + updated_date)
print(type(updated_date))

updated_date: 2022/12/07 18:24
<class 'str'>


In [2]:
##### データフォルダ・データシート作成（データフォルダが存在しない場合のみ実行）
print("###################  データフォルダ・データシート作成  ###################")

current_directory = os.getcwd().replace(os.sep, "/")
print("current_directory: " + current_directory)

data_path = current_directory + "/Data"
print("data_path: " + data_path)

datasheet_all_path = data_path + "/datasheet_all" + ".csv"
print("datasheet_all_path: " + datasheet_all_path)

cols_for_all = ["Data No", "Gonio", "Echo", "Date", "Subject", "Pattern", "Trial Num", "Depth", "RMSE", "R2", "Corrcoef", "Updated Date"]

# データフォルダが存在しなければ作成
if not os.path.exists(data_path):
    os.makedirs(data_path)
    print("Data folder was successfully created.")
else:
    print("Data folder already exists.")

# datasheet_allが存在しなければ作成
if not os.path.exists(datasheet_all_path):
    df_for_all = pd.DataFrame(columns=cols_for_all)
    df_for_all.set_index("Data No", inplace=True)
    # datasheet_allへの新規書き込み
    df_for_all.to_csv(datasheet_all_path, encoding="shift_jis")
    print("datasheet_all was successfully created.")
    
else:
    print("Datasheet already exists.")
print("#####################################################################\n")

###################  データフォルダ・データシート作成  ###################
current_directory: C:/Users/katagi/Desktop/Research/UltrasoundImaging
data_path: C:/Users/katagi/Desktop/Research/UltrasoundImaging/Data
datasheet_all_path: C:/Users/katagi/Desktop/Research/UltrasoundImaging/Data/datasheet_all.csv
Data folder already exists.
Datasheet already exists.
#####################################################################



In [3]:
##### datasheet_all.csv 読み込み
print("###################  datasheet_all.csv 読み込み  ###################")

# インデックスを"Data No"としたデータフレームとして読み込み
datasheet_all_df = pd.read_csv(datasheet_all_path, header=0, index_col=["Data No"], encoding='shift_jis')
datasheet_all_df = datasheet_all_df.dropna(how='all', axis=0)
print("-------------  datasheet_all_df  ---------------------")
print(datasheet_all_df)
print("------------------------------------------------------")

data_No_list = datasheet_all_df.index.tolist()
if datasheet_all_df.empty:
    data_last_No = 0
else:
    data_last_No = data_No_list[-1]
print("data_No_list: " + str(data_No_list))
print("data_last_No: " + str(data_last_No))
print("#####################################################################\n")

###################  datasheet_all.csv 読み込み  ###################
-------------  datasheet_all_df  ---------------------
                    Gonio                 Echo        Date Subject  Pattern  \
Data No                                                                       
1        202210211613.csv  20221021 161310.mp4  20221021.0  Katagi      3.0   
2        202210211616.csv  20221021 161620.mp4  20221021.0  Katagi      3.0   
3        202210211619.csv  20221021 161915.mp4  20221021.0  Katagi      3.0   
4        202210211622.csv  20221021 162244.mp4  20221021.0  Katagi      3.0   
5        202210211626.csv  20221021 162613.mp4  20221021.0  Katagi      3.0   
6        202211081334.csv  20221108 133448.mp4  20221108.0  Katagi      1.0   
7        202211081339.csv  20221108 133918.mp4  20221108.0  Katagi      1.0   
8        202211081343.csv  20221108 134301.mp4  20221108.0  Katagi      1.0   
9        202211081345.csv  20221108 134549.mp4  20221108.0  Katagi      1.0   
10       20

In [4]:
##### DATA_INFO(date, subject, pattern) を入力
print("###################  DATA_INFO(date, subject, pattern) を入力  ###################")
print("DATA_INFOを入力してdatasheet_all.csvに書き込むデータを指定してください．")

## DATA_INFO
# 実験日
date = input("Date: ")

# 被験者
subject = input("Subject: ")

# 実験パターン
pattern = input("Pattern: ")

# 深度
depth = input("Depth: ")


info_str = date + "_" + subject + "_pattern" + pattern
data_per_info_path = data_path + "/" + info_str
print("DATA_INFO: " + info_str)

# data_per_info フォルダが存在しなければ任意で作成
if not os.path.exists(data_per_info_path):
    make_new = input("該当するフォルダがありません．新規作成しますか？(yes/no)")
    if make_new == "yes":
        os.makedirs(data_per_info_path)
        print("\nフォルダを新規作成しました．\n\n")
        print("このフォルダに実験データを入れてからもう一度実行してください．")
        print("※それぞれのフォルダ名は'GonioData'，'EchoData'とすること．")
        raise Exception("Data doesn't exist.")
    else:
        raise Exception("Try again.")
print("#####################################################################\n")

###################  DATA_INFO(date, subject, pattern) を入力  ###################
DATA_INFOを入力してdatasheet_all.csvに書き込むデータを指定してください．


Date:  20221206
Subject:  Katagi
Pattern:  3
Depth:  30


DATA_INFO: 20221206_Katagi_pattern3
#####################################################################



In [5]:
##### データがなければエラーを表示
print("###################  データがなければエラーを表示  ###################")

gonio_data_path = data_per_info_path + "/GonioData"
echo_data_path = data_per_info_path + "/EchoData"
if not os.path.exists(gonio_data_path):
    print("ゴニオデータがありません")
    raise Exception("GonioData doesn't exist.")
if not os.path.exists(echo_data_path):
    print("エコーデータがありません")
    raise Exception("EchoData doesn't exist.")
print("#####################################################################\n")

###################  データがなければエラーを表示  ###################
#####################################################################



In [6]:
## ゴニオ，エコーデータがあればリストで取得
print("###################  ゴニオ，エコーデータがあればリストで取得  ###################")
# ゴニオデータファイルをリストで取得
gonio_files = list()
for file in os.listdir(gonio_data_path):
    if file.endswith(".csv"):
        gonio_files.append(file)

# エコーデータファイルをリストで取得
echo_files = list()
for file in os.listdir(echo_data_path):
    if file.endswith(".mp4"):
        echo_files.append(file)

print("gonio_files: " + str(gonio_files))
print("echo_files: " + str(echo_files))
print("#####################################################################\n")

###################  ゴニオ，エコーデータがあればリストで取得  ###################
gonio_files: ['202212061559.csv', '202212061602.csv', '202212061605.csv', '202212061607.csv', '202212061610.csv']
echo_files: ['20221206 155955.mp4', '20221206 160233.mp4', '20221206 160509.mp4', '20221206 160751.mp4', '20221206 161040.mp4']
#####################################################################



In [7]:
##### datasheet_all.csvへ書き込み
print("###################  datasheet_all.csvへ書き込み  ###################")
# 試行数
num_of_trial = len(gonio_files)
print("num_of_trial: " + str(num_of_trial))

# 書き込み用のデータフレームを用意
new_df_for_write = pd.DataFrame(columns=cols_for_all)
new_df_for_write.set_index("Data No", inplace=True)

# 各試行ごとのデータ取得
for trial in range(num_of_trial):
    # 試行番号
    trial_num = trial + 1
    
    # データ番号
    data_No = data_last_No + trial_num
    
    ## DATA_PATH
    # ゴニオファイル名
    gonio_file = gonio_files[trial]

    # エコーファイル名
    echo_file = echo_files[trial]
    
    # 各試行ごとのデータフレームを作成
    new_data = [(data_No, gonio_file, echo_file, date, subject, pattern, trial_num, depth, "", "", "", updated_date)]
    new_df_element = pd.DataFrame(data=new_data, columns=cols_for_all)
    new_df_element.set_index("Data No", inplace=True)
    
    # 書き込み用データフレームに追加
    new_df_for_write = new_df_for_write.append(new_df_element)

print("-------------  new_df_for_write  ---------------------")
print(new_df_for_write)
print("------------------------------------------------------")

# datasheet_all.csvに書き込み
datasheet_all_df = datasheet_all_df.append(new_df_for_write)
datasheet_all_df.to_csv(datasheet_all_path, encoding="shift_jis")

print("datasheet_all.csvに新しくデータを書き込みました．")
print("#####################################################################\n")

###################  datasheet_all.csvへ書き込み  ###################
num_of_trial: 5
-------------  new_df_for_write  ---------------------
                    Gonio                 Echo      Date Subject Pattern  \
Data No                                                                    
36       202212061559.csv  20221206 155955.mp4  20221206  Katagi       3   
37       202212061602.csv  20221206 160233.mp4  20221206  Katagi       3   
38       202212061605.csv  20221206 160509.mp4  20221206  Katagi       3   
39       202212061607.csv  20221206 160751.mp4  20221206  Katagi       3   
40       202212061610.csv  20221206 161040.mp4  20221206  Katagi       3   

        Trial Num Depth RMSE R2 Corrcoef      Updated Date  
Data No                                                     
36              1    30                   2022/12/07 18:24  
37              2    30                   2022/12/07 18:24  
38              3    30                   2022/12/07 18:24  
39              4    30   